# NLTK


Pada modul ini kita akan mengimplementasikan "Topic MOdeling". Fungsi dari permodelan ini adalah untuk menentukan topic dari sebuah / atau beberapa dokumen. 

Model ini dilakukan dengan asumsi tidak diketahui pengklasifikasian dari dokumen. Dalam hal ini lebih tepatnya kita menggunakan unsupervised.

Untuk dokumen yang sudah diketahui klasifikasinya, dan diperlukan fungsi predictive dapat menggunakan tehnik regresi atau klasifikasi.

## Import Libraries
Untuk mengimplementasikan NLTK pada python, library paling umum digunakan adalah nltk. Untuk memproses dokumen dengan bahasa inggris, library ini sangat cukup, akan tetapi untuk bahasa indonesia kita perlu library lain.

!pip install nltk

In [48]:
import nltk
# nltk.download(["punkt"])

Library CSV akan kita gunakan karena data kita berupa csv.

In [51]:
import csv


Library Sastrawi adalah library yang memiliki kemampuan pemrosesan bahasa indonesia. library ini dapat kita gunakan untuk mengganti kekurangan library nltk dalam memproses bahasa Indonesia

In [52]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

Library pickle akan kita gunakan untuk menyimpan python object ke dalam sebuah file. hal ini diperlukan jika kita mempunya sebuah objek yang merupakan hasil sebuah proses, jika kita mngulangnya membutuhkan waktu yang lama untuk memprosesnya.

In [53]:
import pickle

Library Gensim kita gunakan untuk melaksanakan proses LDA

In [54]:
import gensim
from gensim import corpora

Library pyLDAvis kita panggil untuk melakukan visualisasi data sehingga mudah dimengerti.

In [55]:
import pyLDAvis.gensim

## Load Data
kita load data yang telah disiapkan dalam folder yang sama. Data berupa CSV. Silahkan pahami isi datanya. 

Memahami data adalah salah satu step yang sangan penting dalam proses Machine Learning. 

In [56]:
data = []

with open('artikel.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:  
        data.append(row)
len(data)

101

Cek apakah data sudah tersimpan dengan baik. Data Berupa 100 dokumen. Cek dan pastikan data hanya ada 100.


Buat perubahan yang diperlukan

In [57]:
data = data[1:]

In [58]:
data[0][0].splitlines()

['Api dan Bom Sambut Awal Milenium Baru',
 '',
 'Lima orang tewas dan sekitar 130 cedera, 20 di antaranya mengalami luka bakar serius, ketika Kafe Het Hemeltje (Surga Kecil) di Volendam, kota di utara Belanda, yang dipadati remaja yang tengah merayakan Tahun Baru terbakar. Api bermula setengah jam setelah awal milenium baru, Senin (1/1) pukul 00.30, dengan cepat melalap kafe di kota wisata tersebut.',
 '',
 'Peristiwa tragis di Volendam hanyalah satu dari sekian rangkaian peristiwa tragis, di tengah-tengah kemeriahan masyarakat dunia merayakan hari pertama milenium baru, milenium ketiga pada 1 Januari 2001 dini hari.',
 '',
 'Ledakan bom mengguncang dua gereja di bekas negara Uni Soviet, Tajikistan pada malam menjelang Tahun Baru, Minggu. Kedua bom meledak berselang hanya setengah jam di ibu kota Tajikistan, Dushanbe. Satu bom di Gereja Ortodoks, dan satu lagi di Gereja Adven Hari Ketujuh, tak menimbulkan korban.',
 '',
 'Ini merupakan insiden serupa kedua dalam tiga bulan di negara As

Seperti diketahui data berupa 100 dokumen berita dengan berbagai topik. untuk melakukan LDA kita akan menganalisa paragraf per paragraf. untuk itu kita perlu memisah dokem menjadi paragraf.

Pisahkan list dokumen menjadi list of list dengan struktur list paragraf dalam list dokumen. Jangan lupa cek jumlah dokumen, harus tetap 100.

In [59]:
for i in range (0, len(data)) :
    data[i] = data[i][0].splitlines()

In [60]:
#hasilnya harus 100
len(data)

100

In [61]:
len(data[0])

41

## Steammer
selanjutnya kita lakukan steaming dokumen. Hal ini diperlukan untuk mendapatkan bentuk dasar dari kaliman. 

#### contohnya : 

menghapal --> hapal

membeku --> beku


Proses stemaing ini kita menggunakan library sastrawi.


for corpus in data: 	for kalimat in corpus : 		stemmer.stem(kalimat) 

In [62]:
factory = StemmerFactory()
stemmer = factory.create_stemmer() ## selian stemmer juga ada library lain misalnya limitizer
corpus_dasar = [[stemmer.stem(kalimat) for kalimat in corpus] for corpus in data]

In [63]:
#pastikan datanya benar dan temukan perbedaannya.
corpus_dasar

[['api dan bom sambut awal milenium baru',
  '',
  'lima orang tewas dan sekitar 130 cedera 20 di antara alami luka bakar serius ketika kafe het hemeltje surga kecil di volendam kota di utara belanda yang padat remaja yang tengah raya tahun baru bakar api mula tengah jam telah awal milenium baru senin 1 1 pukul 00 30 dengan cepat lalap kafe di kota wisata sebut',
  '',
  'peristiwa tragis di volendam hanya satu dari sekian rangkai peristiwa tragis di tengah riah masyarakat dunia raya hari pertama milenium baru milenium tiga pada 1 januari 2001 dini hari',
  '',
  'ledak bom guncang dua gereja di bekas negara uni soviet tajikistan pada malam jelang tahun baru minggu dua bom ledak selang hanya tengah jam di ibu kota tajikistan dushanbe satu bom di gereja ortodoks dan satu lagi di gereja adven hari tujuh tak timbul korban',
  '',
  'ini rupa insiden rupa dua dalam tiga bulan di negara asia tengah duduk mayoritas muslim ini turut polisi tempat peristiwa tiga bulan lalu ketika buah bom leda

Proses diatas mengkonsumsi waktu yang cukup lama. Simpan objek ke sebuah file. 

Sehingga apabila kernel kita error kita tidak perlu meload dari awal lg.

In [64]:
with open("corpus_dasar.pkl","wb") as openfile:
    pickle.dump(corpus_dasar, openfile)

## Tokenizer

Tokenisasi adalah proses memecah dokumen majadi satu - persatu kata.

contoh :

" Kita Belajar di pusdik" --> "kita", "belajar", "di", "pusdik"

seperti diperhatikan diatas, selain dipecah, hurus semua dirubah menjadi lower case. Hal ini diperlukan agar huruf Belajar dan belajar diperhitungkan sebagai satu kata. Perlu ditekankan lagi, secara default,  komputer tidak akan bisa mengenali perbedaan lower dan upper case.

Jangan lupa cek hasilnya.

In [65]:
kata = [[nltk.tokenize.word_tokenize(kata) for kata in corpus] for corpus in corpus_dasar]

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/hargi/nltk_data'
    - '/Users/hargi/opt/anaconda3/nltk_data'
    - '/Users/hargi/opt/anaconda3/share/nltk_data'
    - '/Users/hargi/opt/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


## Stopword removal
Kita akan menghilangkan kata - kata yang sangat sering digunakan akan tetapi tidak banyak berarti.

Kata-kata seperti : dan, yang, itu, ini dihilangkan.

Untuk menambah akurasi data, kata yang hanya terdiri dari 2 atau 1 saruf kita hilangkan.

In [39]:
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

In [41]:
kata = [[[each for each in item if each not in stopwords] for item in doc] for doc in kata]
# Hilangkan kata yang terdiri dari dua atau 1 huruf




NameError: name 'kata' is not defined

In [ ]:
bersih = []
for each in kata:
    simpan = []
    for item in each:
        if len(item) > 2 :
            simpan.append(item)
    bersih.append(simpan)

In [ ]:
print(bersih[0])

## Bag of Word
Bag of words (BOW) adalah istilah yang sangat penting dalam proses nlp. Pastikan anda paham dan mengerti konsepnya. Jika tidak, tanyakan kepada instruktur.

In [ ]:
dictionary = corpora.Dictionary(kata[0])

In [ ]:
corpus = [dictionary.doc2bow(text) for text in kata[0]]

Cek pemahaman dengan print output hasilnya dan print kata berkaitan dengan id nya.

gunakan id2word(id)

## LDA
Berikutnya kita akan melakukan modeling. seperti dibahas dalam kelas, LDA adalah salah satu tehnik unsupervised learning. Kelemahan dari unsupervised learning adalah kita tidak banyak informasi tetang model yang tepat.

Dalam hal topic modeling, kita tidak mengetahui berapa topic yang dapat dengan tepat menggambarkan dokumen bersangkutan.

Dalam materi ini kita hanya akan mencakup LDA secara dasar dan cara penggunaannya. Optimalisasi model belum bisa dicakup karena keterbatasan waktu.

Untuk petihan ini kita coba dengan topic sebanyak 5.

In [50]:

NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
#ldamodel.save('model5.gensim')

NameError: name 'corpus' is not defined

Selain jumlah topic, kita juga bisa membatasi jumlah kata dalam tiap tiap topic.

In [ ]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

Untuk lebih memperjelas hasil LDA, kita dapat memvisualisasikan hasilnya.

Visualisasi adalah tahapan penting dalam ML. Hal ini penting karena tidak semua orang paham dengan tehnil ML, akan detatpi dengan visualisasi kita dapat mentejermahkan dengan lebih mudah. Hal ini dikarenakan cara kerja sistem visual manusia yang dapat mengartikan berbagai macam simbol sebagai satu kesatuan informasi. Hal tersebut sangat sudah dicapai oleh mesin canggih sekalipun.

In [ ]:

pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)


In [ ]:
pyLDAvis.display(lda_display)

Berikan penjelasan

# END